# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/lucasludwig/Desktop/Python-API-Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,79.41,72,93,4.79,PF,1680219593
1,1,punta arenas,-53.1500,-70.9167,42.91,61,20,14.97,CL,1680219070
2,2,ushuaia,-54.8000,-68.3000,38.86,70,0,5.75,AR,1680219593
3,3,atuona,-9.8000,-139.0333,80.73,73,4,14.43,PF,1680219593
4,4,mana,5.6592,-53.7767,77.13,86,63,8.90,GF,1680219365


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot from Part 1
city_data_df.hvplot.points(x = "Lon", y = "Lat", hover_cols = "Humidity", geo = True, tiles = "OSM", title = "City Locations vs Humidity", frame_width = 800, frame_height = 500, line_color = "black", line_width = 1, color = "City", size = "Humidity", alpha = 0.8)


# Display the map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that have a max temperature greater than 64 degrees
city_data_df = city_data_df.loc[city_data_df["Max Temp"] >= 65]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(15)

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,79.41,72,93,4.79,PF,1680219593
3,3,atuona,-9.8000,-139.0333,80.73,73,4,14.43,PF,1680219593
4,4,mana,5.6592,-53.7767,77.13,86,63,8.90,GF,1680219365
5,5,cururupu,-1.8283,-44.8683,73.99,98,100,2.28,BR,1680219366
12,12,panuco,22.0500,-98.1667,78.98,60,99,15.32,MX,1680219594
13,13,mount isa,-20.7333,139.5000,82.17,28,19,6.91,AU,1680219594
14,14,cockburn town,21.4612,-71.1419,77.99,75,1,14.25,TC,1680219594
15,15,panaba,21.2833,-88.2667,86.32,42,41,15.48,MX,1680219594
19,19,butaritari,3.0707,172.7902,82.04,79,69,11.59,KI,1680219595
22,22,karratha,-20.7377,116.8463,81.52,75,16,5.23,AU,1680219595


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df and reduce the columns to city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lon", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lon,Humidity,Hotel Name
0,rikitea,PF,-23.1203,-134.9692,72,
3,atuona,PF,-9.8000,-139.0333,73,
4,mana,GF,5.6592,-53.7767,86,
5,cururupu,BR,-1.8283,-44.8683,98,
12,panuco,MX,22.0500,-98.1667,60,
13,mount isa,AU,-20.7333,139.5000,28,
14,cockburn town,TC,21.4612,-71.1419,75,
15,panaba,MX,21.2833,-88.2667,42,
19,butaritari,KI,3.0707,172.7902,79,
22,karratha,AU,-20.7377,116.8463,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lon"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params, headers={"x-api-key": geoapify_key}).json()
    
    # Pretty print the JSON data
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Chez Bianca & Benoit', 'country': 'France', 'country_code': 'fr', 'state': 'French Polynesia', 'city': 'Gambier', 'village': 'Rikitea', 'postcode': '98755', 'district': 'Mangareva', 'street': 'Chemin des Sœurs', 'lon': -134.96602067460267, 'lat': -23.1275846, 'formatted': 'Chez Bianca & Benoit, Chemin des Sœurs, 98755 Gambier, France', 'address_line1': 'Chez Bianca & Benoit', 'address_line2': 'Chemin des Sœurs, 98755 Gambier, France', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Chez Bianca & Benoit', 'osm_id': 751195405, 'tourism': 'hotel', 'osm_type': 'w'}}, 'distance': 814, 'place_id': '5161b68060e9de60c05938ff2327a62037c0f00102f9010d55c62c000000009203144368

,City,Country,Lat,Lon,Humidity,Hotel Name
0,rikitea,PF,-23.1203,-134.9692,72,Chez Bianca & Benoit
3,atuona,PF,-9.8000,-139.0333,73,Pearl Resort
4,mana,GF,5.6592,-53.7767,86,Le Samana
5,cururupu,BR,-1.8283,-44.8683,98,No hotel found
12,panuco,MX,22.0500,-98.1667,60,Hotel Plaza
...,...,...,...,...,...,...
568,pisco,PE,-13.7000,-76.2167,73,La Portada
569,sehore,IN,23.2000,77.0833,66,No hotel found
572,sokoni,TZ,-6.4333,39.5500,76,La Madrugada Beach Hotel
574,sungairaya,ID,0.7000,108.9000,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(x = "Lon", y = "Lat", hover_cols = ["Hotel Name", "Country"], geo = True, tiles = "OSM", title = "Hotel Locations", frame_width = 800, frame_height = 500, color = "City", size = "Humidity",line_color = "black", line_width = 1, alpha = 0.8, scale = 1)

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Humidity,Hotel Name,Country)